In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import GoogleGenerativeAI
from typing import Annotated, TypedDict
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
llm = GoogleGenerativeAI(model='gemini-2.5-flash')

E0000 00:00:1759474646.845166  104020 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [3]:
class BlogState(TypedDict):
  title: Annotated[str, 'This is title of the blog']
  outline: Annotated[str, 'This is outline of the blog']
  content: Annotated[str, 'This is content of the blog']

In [11]:
def create_outline(state:BlogState) ->BlogState:
  
  title = state['title']
  
  prompt = f'generate a 3 one sentence points outline for the blog of the title: {title}'
  
  outline = llm.invoke(prompt)

  state['outline'] = outline
  
  return state

In [12]:
def create_content(state: BlogState)-> BlogState:
  
  title = state['title']
  outline = state['outline']
  
  prompt = f"create a 100 words short blog from the given outline and title \n\ntitle:{title}\n\noutline:{outline}"
  
  content = llm.invoke(prompt)
  
  state['content'] = content
  
  return state

In [13]:
graph_builder = StateGraph(BlogState)

graph_builder.add_node('create_outline', create_outline)
graph_builder.add_node('create_content', create_content)

graph_builder.add_edge(START, 'create_outline')
graph_builder.add_edge('create_outline', 'create_content')
graph_builder.add_edge('create_content', END)

workflow = graph_builder.compile()

In [14]:
input_state = {'title': "different types of water tastes"}

final_state = workflow.invoke(input_state)

final_state

{'title': 'different types of water tastes',
 'outline': 'Here\'s a 3 one-sentence points outline for a blog titled "Different Types of Water Tastes":\n\n1.  **Natural Sources & Mineral Content:** Different water sources, like natural springs, wells, or municipal taps, impart unique mineral compositions and dissolved solids that define their inherent taste profiles, from crisp and clean to earthy or slightly sweet.\n2.  **Treatment & Filtration Processes:** Chemical treatments, such as chlorination, and various filtration methods significantly alter water\'s taste, often resulting in distinct flavors like a "clean" but sometimes noticeable chemical undertone or a neutral, bland profile.\n3.  **Environmental Factors & Contaminants:** External factors like plumbing materials, dissolved gases, or the presence of contaminants (e.g., metals, algae, organic matter) can introduce undesirable metallic, sulfuric, or musty "off-tastes."',
 'content': '## Different Types of Water Tastes\n\nEver n